In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

In [8]:
input_set_labeled = np.genfromtxt ('training.csv', delimiter=",")
input_set=input_set_labeled[1:input_set_labeled.shape[0],:]
input_data=input_set[:,1:input_set.shape[1]]
target_data = input_set[:,0]
target_data=target_data.reshape(target_data.shape[0],1)
print(input_data.shape)
print(target_data.shape)
print(input_set.shape[1])
print(input_set.shape[0])

(21000, 784)
(21000, 1)
785
21000


In [0]:
#input_data=tf.transpose(input_data)
# sess = tf.Session()
# new_input_data=tf.concat([input_data,target_data],1)
# new_input_data=tf.random_shuffle(new_input_data)
# init_var = tf.global_variables_initializer()
# sess.run(init_var)
# print(new_input_data.shape)
# new_input_data=sess.run(new_input_data)

In [0]:
# input_data=new_input_data[:,0:len(new_input_data[0])-5]
# target_data=new_input_data[:,len(new_input_data[0])-4:len(new_input_data[0])-1]
# print(input_data.shape)
# print(input_data.shape)
# input_data_trans = tf.transpose(input_data)
# target_data_trans = tf.transpose(target_data)
#input_data_min=input_data[:,0:len(input_data[0])]
# print(target_data_trans.shape)

In [11]:
sess = tf.Session()
init_var = tf.global_variables_initializer()
sess.run(init_var)
indices1 = tf.cast(target_data, tf.int32)
one_hot_vecs1 = tf.one_hot(indices1,np.max(sess.run(indices1))+1)
new_output=sess.run(one_hot_vecs1)
new_output=new_output.reshape(21000,10)
print(new_output)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [12]:
input_train,input_test, target_train, target_test =train_test_split(input_data,new_output,test_size=0.166667, random_state=10)
print(input_train.shape)
print(input_test.shape)
print(target_train.shape)
print(target_test.shape)

(17499, 784)
(3501, 784)
(17499, 10)
(3501, 10)


In [13]:
input_train_map=input_train.reshape(-1,28,28,1)
input_test_map=input_test.reshape(-1,28,28,1)
print(input_train_map.shape)
print(input_test_map.shape)

(17499, 28, 28, 1)
(3501, 28, 28, 1)


In [0]:
# model = Sequential()
# #32 core，with size 3*3
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# #2*2 max pool
# model.add(MaxPooling2D(pool_size=(2, 2)))
# #drop out layor
# #model.add(Dropout(0.25))
# #64 core，with size 3*3
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# #2*2 max pool
# model.add(MaxPooling2D(pool_size=(2, 2)))
# #drop out layor
# #model.add(Dropout(0.25))
# #flatten layor
# model.add(Flatten())
# #full connected
# model.add(Dense(512, activation='relu'))
# #model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.5))


# #sorting
# model.add(Dense(10, activation='softmax'))

# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='categorical_crossentropy', optimizer=sgd)



In [23]:
# kfold
#X,X_test, y, y_test =train_test_split(new_input_data,target_data,test_size=0.1667, random_state=10)
# X_train,X_validation, y_train, y_validation =train_test_split(X,y,test_size=0.2, random_state=10)
score=[]
kf = KFold(n_splits=5)
index=0
for train_index, test_index in kf.split(input_train):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test_loop = input_train_map[train_index], input_train_map[test_index]
    y_train, y_test_loop = target_train[train_index], target_train[test_index]
    print(X_train.shape)
    model = Sequential()
    #32 core，with size 3*3
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    #2*2 max pool
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #drop out layor
    #model.add(Dropout(0.25))
    #64 core，with size 3*3
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    #2*2 max pool
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #drop out layor
    #model.add(Dropout(0.25))
    #flatten layor
    model.add(Flatten())
    #full connected
    model.add(Dense(512, activation='relu'))
    #model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    #sorting
    model.add(Dense(10, activation='softmax'))
    sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd)
    filepath=str(index)+'weights.best.hdf5'
    index=index+1
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True,
                            mode='auto')
    callbacks_list = [checkpoint]
    model.fit(X_train, y_train, batch_size=96, epochs=100,callbacks=callbacks_list,verbose=0)
#     model.fit(X_train, y_train,  batch_size=147, epochs=1000,callbacks=callbacks_list,verbose=0)
    score.append(model.evaluate(X_test_loop, y_test_loop, batch_size=147))
    

(13999, 28, 28, 1)

Epoch 00001: loss improved from inf to 2.01207, saving model to 0weights.best.hdf5

Epoch 00002: loss improved from 2.01207 to 0.24151, saving model to 0weights.best.hdf5

Epoch 00003: loss improved from 0.24151 to 0.15857, saving model to 0weights.best.hdf5

Epoch 00004: loss improved from 0.15857 to 0.12089, saving model to 0weights.best.hdf5

Epoch 00005: loss improved from 0.12089 to 0.10284, saving model to 0weights.best.hdf5

Epoch 00006: loss improved from 0.10284 to 0.08281, saving model to 0weights.best.hdf5

Epoch 00007: loss improved from 0.08281 to 0.07510, saving model to 0weights.best.hdf5

Epoch 00008: loss improved from 0.07510 to 0.05761, saving model to 0weights.best.hdf5

Epoch 00009: loss improved from 0.05761 to 0.05459, saving model to 0weights.best.hdf5

Epoch 00010: loss improved from 0.05459 to 0.04801, saving model to 0weights.best.hdf5

Epoch 00011: loss improved from 0.04801 to 0.04326, saving model to 0weights.best.hdf5

Epoch 00012: los

In [24]:
print("score=: ",score)
print("average score in five loop: ",np.mean(score))
final_score=[]
for iteration in range (0,5):
  filepath=str(iteration)+'weights.best.hdf5'
#   print(filepath)
  model.load_weights(filepath)
  model.save(str(iteration)+"CNN.model")
  final_score.append(model.evaluate(input_test_map, target_test, batch_size=147))
# final_score=model.evaluate(X_test, y_test, batch_size=147)
print(final_score)
print("best score=: " ,np.min(final_score))
# print(X_validation)

score=:  [0.07094797687047685, 0.08565460482053458, 0.05898138884175569, 0.059298772442911284, 0.0647477513330666]
average score in five loop:  0.067926098861749
3501/3501 [==============================] - 0s 52us/step
[0.06447665215004884, 0.07287929850366633, 0.06034064279262333, 0.06046174851295819, 0.06559375124701454]
best score=:  0.06034064279262333


In [20]:
#model.fit(input_train_map, target_train, batch_size=96, epochs=100)


Instructions for updating:
Use tf.cast instead.
Epoch 1/100
17499/17499 [==============================] - 5s 297us/step - loss: 1.3369
Epoch 2/100
17499/17499 [==============================] - 2s 138us/step - loss: 0.2025
Epoch 3/100
17499/17499 [==============================] - 2s 138us/step - loss: 0.1366
Epoch 4/100
17499/17499 [==============================] - 2s 139us/step - loss: 0.1083
Epoch 5/100
17499/17499 [==============================] - 2s 138us/step - loss: 0.0900
Epoch 6/100
10848/17499 [=================>............] - ETA: 0s - loss: 0.0854

KeyboardInterrupt: ignored

In [0]:
# score = model.evaluate(input_test_map, target_test, batch_size=8)
# print(score)
# plot_model(model, to_file='modelcnn.png',show_shapes=True)


In [0]:
# input_data_test_finish = np.genfromtxt('data_test.csv', delimiter=",")
# input_data_min=input_data_test_finish[0:len(input_data_test_finish)-1]
# input_data_map=input_data_min.reshape(-1,640,480,1)
# predict_test=model.predict(input_data_map)
# print(predict_test)
# predict_test_array=predict_test.tolist()
# index=predict_test_array[0].index(max(predict_test_array[0]))
# if index==0:
#   print("I saw a rectangle")
# elif index==1:
#   print("I saw a gear")
# else:
#   print("It is not gear nor rectangle")

In [0]:
model.save("CNN.model")

In [0]:
model.save('my_model.h5')

In [0]:
model.save_weights('my_checkpoint')